A program that reads and processes images for a Convolutional Neural Network (CNN) to classify as images as good or bad.

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras import optimizers
from keras import callbacks
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.misc import imresize
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.
C:\Users\micha\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def read_images(paths): 
    """
    Reads in all images and returns list of picture id numbers based on the image name
    
    Parameters
    ----------
    paths : string
    
    Returns
    ----------
    images and list of id numbers
    """
    #Get list of images
    images = (glob(paths + '*.jpg'))
    #Read images from list
    data = [cv2.imread(file) for file in images]

    return data

In [3]:
def process_images(images, size = 60):
    """
    Import image at 'paths', center and crop to size
    Code from https://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb
    """

    count = len(images)
    arr = np.zeros(shape=(count,size,size,3))
    for i in range(count):
        img = images[i]
        img = img.T[:,106:106*3,106:106*3] #crop 424x424x3 to 212x212x3
        img = imresize(img,size=(size,size,3),interp="cubic") # shrink size to make easier to compute
        arr[i] = img

    return arr.astype(int)

In [4]:
def scale_features(X):
    '''
    input: X (np array of any dimensions)
    cast as floats for division, scale between 0 and 1
    output: X (np array of same dimensions)
    '''
    X = X.astype("float32")
    X /= 255
    return X

In [5]:
def cnn_layers(x_train, y_train, x_test, y_test, batch_size = 4, nb_classes = 2, nb_epoch = 20, input_size = (60,60, 3)):
    """
    Builds layers of Convolutional Neural Net
    Fits model to the data
    
    Parameters
    ------------
    x_train = array
    x_test = array
    y_train = data frame or array
    y_test = data frame or array
    batch_size = integer
    nb_classes = integer
    nb_epoch = integer
    input_size = list
    
    Returns
    ------------
    model metrics evaluation
    """
    
    model = Sequential()
        
    #first convolutional layer and pooling
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(input_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #second convolutional layer and pooling
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(input_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #flatten images
    model.add(Flatten())
    
    #first dense layer
    model.add(Dense(32, init='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    #second dense layer
    model.add(Dense(32, init='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    #output layer
    model.add(Dense(nb_classes, init='glorot_normal'))
    model.add(Activation('softmax'))
    
    #initializes optimizer SGD
    #need to see which learning rate (lr) achieves best results
    sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])
    
    #early stopping batch = x_train, y_train, x_test, y_test
    #need to experiment with patience 
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
    
    checkpointer = callbacks.ModelCheckpoint(filepath=('checkpoint.hdf5'), verbose=1, save_best_only=True)
    
    #     hist = callbacks.History()
    
    model.fit(x_train, y_train, verbose=2, callbacks = [early_stopping, checkpointer], batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(x_test, y_test))
    
    return model, model.evaluate(x_test, y_test, verbose=1)

In [6]:
def convert_targets(targets):
    return pd.get_dummies(targets).values

In [7]:
df_train = pd.read_csv('/Users/micha/ea-applications/data/test-images.csv')
print(df_train.head())
paths = '/Users/micha/ea-applications/data/training-test-images/'

train_images = read_images(paths)
train_arr = process_images(train_images)

# y = df_train.drop(['Image_Name'], axis=1)
y = np.array(df_train['Label'])

# enc = OneHotEncoder(categorical_features=2, handle_unknown='ignore')
# enc.fit(y)

# y = df_train['Label'].values

y = convert_targets(y)
x_train, x_test, y_train, y_test = train_test_split(train_arr, y, random_state=42, test_size=0.2)

print(y_train)

                          Image_Name  Label
0             BIRD_806937_Therm_1104      1
1          CROSSBOW_707661_Therm_657      1
2   DEERFIELD_BEACH_703541_Therm_377      1
3  DEERFIELD_BEACH_703542_Therm_2718      1
4           HARRIS_203635_Therm_1175      1
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]]


C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


In [8]:
model, metrics = cnn_layers(x_train, y_train, x_test, y_test, batch_size = 4, nb_classes = 2, nb_epoch = 10)
print(metrics)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(60, 60, 3..., activation="relu", padding="valid")`
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(60, 60, 3..., activation="relu", padding="valid")`
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, kernel_initializer="glorot_normal")`
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, kernel_initializer="glorot_normal")`
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="glorot_normal")`
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel

Train on 13 samples, validate on 4 samples
Epoch 1/10
 - 0s - loss: 9.0727 - acc: 0.3077 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00001: val_loss improved from inf to 4.02952, saving model to checkpoint.hdf5
Epoch 2/10
 - 0s - loss: 7.4391 - acc: 0.5385 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00002: val_loss did not improve from 4.02952
Epoch 3/10
 - 0s - loss: 4.9594 - acc: 0.6923 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00003: val_loss did not improve from 4.02952
Epoch 4/10
 - 0s - loss: 11.1587 - acc: 0.3077 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00004: val_loss did not improve from 4.02952
Epoch 5/10
 - 0s - loss: 9.9188 - acc: 0.3846 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00005: val_loss did not improve from 4.02952
Epoch 6/10
 - 0s - loss: 8.6790 - acc: 0.4615 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00006: val_loss did not improve from 4.02952
Epoch 7/10
 - 0s - loss: 7.4391 - acc: 0.5385 - val_loss: 4.0295 - val_acc: 0.7500

Epoch 00007: val_loss did not